In [17]:
import pandas as pd
import requests
from sec_api import *
import csv

In [18]:
headers = {"User-Agent": "teng030918@gmail.com"}

In [19]:
print("'University of California, Irvine'")

'University of California, Irvine'


In [20]:
import requests
import time

# Define the API endpoint
api_endpoint = "https://api.sec-api.io/full-text-search"

page = 0
files = {}

# Define your search parameters
search_params = {
    "query": '"University of California, Irvine"',  # Example search query
    # "formTypes": ["8-K", "10-Q"],  # Example form types
    "startDate": "2023-01-01",
    "endDate": "2024-05-04",
}

# Set the API key in the request headers
headers = {
    "Authorization": "7933988e6af8641e0b8c997c7a32e8e80102a086909148434387bd899a3d1638"
}

timeout_seconds = 600
start_time = time.time()


while True:
        if time.time() - start_time > timeout_seconds:
            print("Timeout reached. Exiting loop.")
            break
        
        search_params["page"] = page
        
        response = requests.post(api_endpoint, json=search_params, headers=headers)
        
        # Check if the response is successful
        if response.status_code != 200:
            print("Error:", response.status_code)
            break
            
        # Parse the JSON response
        data = response.json()
        
        # Extract filings information
        filings = data.get("filings", [])
        # Print the filing details
        for filing in filings:
            if filing["cik"] not in files.keys():
                files[filing["cik"]] = []
                files[filing["cik"]].append(filing)
            else:
                files[filing["cik"]].append(filing)
            
        if not filings:
            break
        if len(filings) < 100:
            break

        page += 1

In [21]:
print(len(files.keys()))

283


In [23]:
import csv
with open("sec_outputs.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["CIK", "accession number","URL", "company", "ticker", "description", "form type", "type", "filed at"])

In [24]:
with open("sec_outputs.csv", 'a') as f:
    writer = csv.writer(f)
    for cik in files.keys():
        file_list = files[cik]
        for file in file_list:
            writer.writerow([
                file["cik"],
                file["accessionNo"],
                file["filingUrl"],
                file["companyNameLong"],
                file["ticker"],
                file["description"],
                file["formType"],
                file["type"],
                file["filedAt"]
            ])